In [5]:
 #Requirements :
!pip install praw
!pip install xmltodict


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Le sujet choisi est : Machine Learning

Partie 1  :

In [6]:
# Importer le module
import Document

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(Document)

<module 'Document' from 'c:\\Users\\albar\\Desktop\\TD-Python\\Projet_Python\\TD7\\TD7\\Document.py'>

In [7]:
#Reddit
import praw
from datetime import datetime
import Author

#instanciation de l'authentification aux outils API de Reddit
reddit = praw.Reddit(client_id='WhOeUX8xCa_LSqqzHogeNA', client_secret='IouRe-5putFpKpyr11rOW7Wzh2Rpmw', user_agent='Web Scrapping')

#Les 10 posts les plus tendance du subreddit MachineLearning :
hot_posts = reddit.subreddit('MachineLearning').hot(limit=10)

#variables principales
docs = []                 #stocke les données d'un article (indice 0 → titre, 1 → auteur, 2 → date de publication, 3 → url, 4 → contenu textuel)
origines = []             #tableau indiquant l'origine du document (Reddit ou Arxiv)
id2doc = {}               #Clés : id du document, Valeurs : objet Document
id2aut = {}               #Clés : noms des auteurs, Valeurs : id des documents publiés
id = 1                    #id du document

#pour chaque post Reddit
for posts in hot_posts:
  docs.append(posts.title)                #ajout du titre dans docs
  auteurs = posts.author.name             #ajout du nom d'auteur dans docs
  docs.append(auteurs)

  texte = posts.selftext                  #contenu texte
  texte = texte.replace("\n", " ")                #formatage du texte pour remplacer les sauts de ligne \n par un espace
  texte = texte.replace("\t", " ")
  texte = texte.replace(";", " ")
  texte = texte.replace("&#x200B", "")
  if texte == "":                         #Pour les contenu textuels vides, ils seront rreprésentés par un espace afin de pouvoir être traités par les fonctions du corpus
    texte = " "


  if auteurs not in id2aut.keys():        #vérification de la présence de l'auteur dans id2aut
    id2aut[auteurs] = Author.Author(auteurs)  #instanciation de l'objet Author avec le nouvel auteur
    id2aut[auteurs].add(id, texte)
  else:
    id2aut[auteurs].add(id, texte)           #ajout du document à la production de l'auteur

  dateP = posts.created_utc               #ajout de la date dans docs au format unix
  docs.append(datetime.fromtimestamp(dateP))

  docs.append(posts.url)                  #ajout de l'url dans docs

  docs.append(texte)                      #ajout du contenu texte dans docs
  docs.append(posts.num_comments)         #ajout du nombre commentaire dans docs

  docs.append("Reddit")                   #récupération de l'origine du post

  #Une fois toutes les données du post récupérées, on instancie l'objet Document dans id2doc avec un id unique
  id2doc[id] = Document.RedditDocument(docs[0], docs[1], docs[2], docs[3], docs[4], docs[5], docs[6])
  id+=1                                   #incrémentation de l'id
  docs = []                               #on vide le tableau pour pouvoir ajouter les données du document suivant


Champs disponibles pour Reddit API :
* 'title' (titre du post)
* 'score' (nombre d'upvotes)
* 'id' (identifiant du post, contenu dans l'url)
* 'subreddit' (communauté)
* 'url' (lien dans le post)
* 'num_comments' (nb commentaires)
* 'body' (contenu commentaire)
* 'selftext' (contenu du post si c'est textuel)
* 'author' (nom de l'utilisateur auteur du post)
* created_utc (heure de publication du post)

In [8]:
#Arxiv
import requests
import xmltodict
import dateutil.parser as dp

#url contenant la requete (ici, 10 articles résultant de la recherche des mots Machine et Learning).
url = 'http://export.arxiv.org/api/query?search_query=all:machine+learning&start=0&max_results=10'

data = requests.get(url)                      #Obtention des données au format XML à partir de la requête
articles = xmltodict.parse(data.text)         #données converties du format XML à dictionnaire

#parcours des articles
entries = articles['feed']['entry']
for entry in entries:
  a2 = [] #tableau des coauteurs
  docs.append(entry.get("title"))             #ajout du titre dans docs

  summary = entry['summary']                  #contenu textuel
  summary = summary.replace("\n", " ")
  summary = summary.replace("\t", " ")                  #formatage du texte pour remplacer les sauts de ligne \n par un espace
  summary = summary.replace(";", " ")
  summary = summary.replace("&#x200B", "")

  authors = entry.get("author", [])           #ajout de l'auteur dans docs
  if type(authors) is dict:                   #si un seul auteur
    a = authors.get("name") #a = premier auteur
    if a not in id2aut.keys():                #vérification de la présence de l'auteur dans id2aut
      id2aut[a] = Author.Author(a)  #instanciation de l'objet AUthor avec le nouvel auteur
      id2aut[a].add(id, summary)
    else:
      id2aut[a].add(id, summary)
    docs.append(a)
  else:                                       #si plusieurs auteurs
    docs.append(authors[0].get("name"))
    for au in authors:
      a2.append(au.get("name"))
      if au.get("name") not in id2aut.keys(): #vérification de la présence de l'auteur dans id2aut
        id2aut[au.get("name")] = Author.Author(a2)
        id2aut[au.get("name")].add(id, summary)
      else:
        id2aut[au.get("name")].add(id, summary)
    a2 = a2[1:]                              #on met la lste des co-auteurs dans a2 (donc tous sauf le premier element)

  dateP = entry.get("published")              #ajout de la date de publication dans docs au format unix
  dateP = dp.parse(dateP)
  dateP = dateP.timestamp()
  docs.append(datetime.fromtimestamp(dateP))

  url = entry.get("id")                       #ajout de l'url dans docs
  docs.append(url)

  docs.append(summary)                        #ajout du contenu textuel dans docs

  docs.append(a2)

  docs.append("Arxiv")                        #récupération de l'origine du post

  #Une fois toutes les données du post récupérées, on instancie l'objet Document dans id2doc avec un id unique
  id2doc[id] = Document.ArxivDocument(docs[0], docs[1], docs[2], docs[3], docs[4], docs[5], docs[6])
  docs = []                                   ##on vide le tableau pour pouvoir ajouter les données du document suivant
  id+=1                                       #incrémentation de l'id

Champs disponibles pour Arxiv :

* author/name (nom d'un auteur de l'article)
* title (titre de l'article)
* id (identifiant de l'article)
* published (date de première publication)
* link (lien de l'article)
* update (date de mise à jour de l'article)
* summary (sommaire du contenu de l'article)


Classe Document :

In [9]:
#test avec le 5ème document sur 20
d1 = id2doc[5]
print(d1.type)   #affiche les attributs de l'objet

Reddit


Classe Author :

In [10]:
print("Liste complète des auteurs :")                                #affiche la liste des noms d'auteurs
#tri des auteurs par ordre alphabétique
auts = []
for key in id2aut:
  auts.append(key)
auts.sort()

#affichage de la liste
for elt in auts:
  print("-", elt)

verif = False
print()
while verif is False:                                                #boucle pour que l'utilisateur entre un nom d'auteur valide
  nom  = str(input("Entrez un nom d'auteur parmis ceux proposés : "))
  if nom in id2aut.keys():
    verif = True
  else:
    verif = False
    print("Veuillez entrer un nom d'auteur valide parmi la liste proposée.")
    print()

a1 = id2aut[nom]                                             #instanciation de l'auteur choisi
a1.get_taille_moyenne_documents()                            #affichage des informations concernées

Liste complète des auteurs :
- Alejandro Guerra-Manzanares
- Anindya Iqbal
- AutoModerator
- Binxin Ru
- Catholijn M. Jonker
- Cedric De Boom
- CommunityTough1
- Cool-Statistician880
- Dario Garcia-Gasulla
- David J Wales
- Davide Cacciarelli
- Diego Granziol
- Dmytro Chernyshov
- Dmytro Fishman
- Dmytro Sytnikov
- Emergency-Cobbler137
- Farah E. Shamout
- Felix Mohr
- Foreign_Fee_5859
- Fotis E. Psomopoulos
- Gianluca Pollastri
- Gias Uddin
- Ian Walsh
- Jan N. van Rijn
- Jen Harrow
- Joost Broekens
- Junaed Younus Khan
- L. Julian Lechuga Lopez
- Maximilian P Niroomand
- Md. Tawkat Islam Khondaker
- Michael Osborne
- Michael Reusens
- Michail Maniatakos
- Murat Kulahci
- Norqj
- Olga Cherednichenko
- PhotographOld9150
- Polina Sytnikova
- Sadia Afroz
- Silvio C. E. Tosatto
- Stefan Zohren
- Stephen Roberts
- Substantial_Ring_895
- The ELIXIR Machine Learning focus group
- Thomas M. Moerland
- Tiina Titma
- Xiaowen Doing
- tensorpool_tycho



'Nombre de documents produits : 1 - Taille moyenne des documents : 1394 caractères'

In [11]:
# Importer le module
import Corpus

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(Corpus)

<module 'Corpus' from 'c:\\Users\\albar\\Desktop\\TD-Python\\Projet_Python\\TD7\\TD7\\Corpus.py'>

In [27]:
#Instanciation de corpus
corpus = Corpus.Corpus("Machine Learning", id2aut, id2doc)
print(corpus)
print()
corpus.afficher_par_date(5)
print()
corpus.afficher_par_titre(5)
print()

Corpus 'Machine Learning' : 20 documents, 48 auteurs

Tri par date :
- 2025-11-25 | [P] Knowledge Distillation: 97% Cost Reduction Distilling Claude Sonnet 4 → GPT-4.1-nano (98% Fidelity Retained) (Emergency-Cobbler137)
- 2025-11-25 | [D] I built a reasoning pipeline that boosts 8B models using structured routing + verification (Cool-Statistician880)
- 2025-11-25 | [R] Novel Relational Cross-Attention appears to best Transformers in spatial reasoning tasks (CommunityTough1)
- 2025-11-25 | [R] is there a way to decide on a model architecture using pruning without using NAS? (PhotographOld9150)
- 2025-11-25 | [P] Feedback/Usage of SAM (Segment Anything) (Norqj)

Tri par titre
- [D] I built a reasoning pipeline that boosts 8B models using structured routing + verification (Cool-Statistician880, 2025-11-25)
- [D] ML conferences need to learn from AISTATS (Rant/Discussion) (Foreign_Fee_5859, 2025-11-23)
- [D] Monthly Who's Hiring and Who wants to be Hired? (AutoModerator, 2025-10-31)
- [D] 

Tester le singleton

In [13]:
corpus2 = Corpus.Corpus("Machine_Learning_2", id2aut, id2doc)
print(corpus is corpus2) # retourne True si la même instance est crée (verifie qu'il est singleton)

True


Générateur de documents grâce à un patron de conception d’usine (factory pattern)

In [14]:
# Importer le module
import DocumentFactory

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(DocumentFactory)

<module 'DocumentFactory' from 'c:\\Users\\albar\\Desktop\\TD-Python\\Projet_Python\\TD7\\TD7\\DocumentFactory.py'>

Tester le generateur avec reddit et arxiv

In [15]:
from DocumentFactory import DocumentFactory  # importe la classe

#créer un nouveau document Arxiv
id2doc[id] = DocumentFactory.createDocument("arxiv", titre="Le sujet du monde IA", auteur="Aissatou",
                                            date=datetime.today(),url="https://arxiv.com/xyz",
                                            texte="loremmmmmmmmmm, je fais le tour du monde Texte du post.lom..",
                                            extra=["Bob", "Charlie"])
id+=1                                       #incrémentation de l'id

#créer un nouveau document Reddit
id2doc[id] = DocumentFactory.createDocument("reddit", titre="Le tour du monde IA", auteur="Barry",
                                            date=datetime.today(),url="https://reddit.com/xyz",
                                            texte="je fais le tour du monde Texte du post.lom..",
                                            extra=50)
id+=1                                       #incrémentation de l'id

In [16]:
d2 = corpus.id2doc[5]     #test avec le dernier document créé
print(d2)   #affiche les attributs de l'objet

Titre : [R] Novel Relational Cross-Attention appears to best Transformers in spatial reasoning tasks
Auteur : CommunityTough1
Date : 2025-11-25
Url : https://www.reddit.com/r/MachineLearning/comments/1p6cz0f/r_novel_relational_crossattention_appears_to_best/
Texte : Repo (MIT): [https://github.com/clowerweb/relational-cross-attention](https://github.com/clowerweb/r...
Nombre de commentaires : 0
Type : Reddit


In [17]:
#Enregistre le corpus dans un fichier
corpus.save("data.csv")

#Charge un fichier csv pour créer Dataframe du corpus
newcorps = corpus.load('data.csv')
print(newcorps)

texte_join=""
#Créé une chaine de caractères unique contenant tous les documents grâce à la fonction join et le séparateur " "
for texte in newcorps.id2doc.values():
  texte_join += texte.texte #rassemble le texte de chaque document

print('La Chaine de caractere unique contenant tous les documents : \n', texte_join)

Corpus 'Machine Learning' : 20 documents, 48 auteurs
La Chaine de caractere unique contenant tous les documents : 
 Please post your personal projects, startups, product placements, collaboration needs, blogs etc.  Please mention the payment and pricing requirements for products and services.  Please do not post link shorteners, link aggregator websites , or auto-subscribe links.  \--  Any abuse of trust will lead to bans.  Encourage others who create new posts for questions to post here instead!  Thread will stay alive until next one so keep posting after the date in the title.  \--  Meta: This is an experiment. If the community doesnt like this, we will cancel it. This is to encourage those in the community to promote their work by not spamming the main threads.**For Job Postings** please use this template  >Hiring: \[Location\], Salary:\[\], \[Remote | Relocation\], \[Full Time | Contract | Part Time\]    and \[Brief overview, what you're looking for\]  **For Those looking for jobs*

In [18]:
# tester la recherche d'un mot
corpus.search(texte_join, "AI")

# tester concorde
corpus.concorde(texte_join, "AI", 30)

1 : e getting low quality or even AI written reviews. While this i
2 :  even a small chance of being AI written (i think everyone sho
3 : owards widespread adoption of AI in highly sensitive areas suc
4 : tion theories in a successful AI agent class. This survey prov
                    contexte gauche motif trouvé                    contexte droit
0  ...e getting low quality or even           AI  written reviews. While this i...
1  ... even a small chance of being           AI  written (i think everyone sho...
2  ...owards widespread adoption of           AI  in highly sensitive areas suc...
3  ...tion theories in a successful           AI  agent class. This survey prov...


2.1 et 2.2

In [19]:
# Tester les mots les plus frequents
corpus.stats()

Nombre de mots différents : 1236
Les 10 mots les plus fréquents :
        mot  term frequency  document frequency
0       the             132                  17
1        of             104                  18
2       and              98                  19
3        to              90                  18
4         a              66                  17
5        in              55                  17
6  learning              47                  12
7       for              43                  17
8      with              34                  15
9      data              32                  11


In [25]:
vocab = corpus.freq_mots()
vocab = vocab[0]
print(vocab)

{'please': {'id': 1, 'occurences': 6, 'docFreq': np.int64(2)}, 'post': {'id': 2, 'occurences': 6, 'docFreq': np.int64(4)}, 'your': {'id': 3, 'occurences': 3, 'docFreq': np.int64(3)}, 'personal': {'id': 4, 'occurences': 1, 'docFreq': np.int64(1)}, 'projects': {'id': 5, 'occurences': 1, 'docFreq': np.int64(1)}, 'startups': {'id': 6, 'occurences': 1, 'docFreq': np.int64(1)}, 'product': {'id': 7, 'occurences': 1, 'docFreq': np.int64(1)}, 'placements': {'id': 8, 'occurences': 1, 'docFreq': np.int64(1)}, 'collaboration': {'id': 9, 'occurences': 1, 'docFreq': np.int64(1)}, 'needs': {'id': 10, 'occurences': 1, 'docFreq': np.int64(1)}, 'blogs': {'id': 11, 'occurences': 2, 'docFreq': np.int64(2)}, 'etc': {'id': 12, 'occurences': 1, 'docFreq': np.int64(1)}, 'mention': {'id': 13, 'occurences': 1, 'docFreq': np.int64(1)}, 'the': {'id': 14, 'occurences': 132, 'docFreq': np.int64(17)}, 'payment': {'id': 15, 'occurences': 1, 'docFreq': np.int64(1)}, 'and': {'id': 16, 'occurences': 98, 'docFreq': np.in

In [23]:
vocab = corpus.freq_mots()
vocab = vocab[0]
corpus.moteurRecherche(vocab)

Tappez 'stop' pour terminer la recherche...

5 Résultats :


Val : 0.10332787555729644


Val : 0.05581127454196056


Val : 0.04021448898487182


Val : 0.03613862028589827


Val : 0.03169351454602337


In [39]:
# Importer le module
import SearchEngine

# Recharger le module si nécessaire (utile si déjà importé avant)
import importlib
importlib.reload(SearchEngine)

<module 'SearchEngine' from 'c:\\Users\\albar\\Desktop\\TD-Python\\Projet_Python\\TD7\\TD7\\SearchEngine.py'>

In [40]:
# Instancier un moteur de recherche sur le corpus qu'on avait deja en amont(Machine Learning)
search = SearchEngine.SearchEngine(corpus)

In [ ]:
#Faire la recherche
search.recherche()

Tappez 'stop' pour terminer la recherche...
   id                                              titre        date     score                                              texte
0  22                                Le tour du monde IA  2025-11-25  0.209061       je fais le tour du monde Texte du post.lom..
1  21                               Le sujet du monde IA  2025-11-25  0.112922  loremmmmmmmmmm, je fais le tour du monde Texte...
2  14         Active learning for data streams: a survey  2023-02-17  0.070499  Online active learning is a paradigm in machin...
3  11  Changing Data Sources in the Age of Machine Le...  2023-06-07  0.049056  Data science has become increasingly essential...
4   9  [R] Struggle with PaddlePaddle OCR Vision Lang...  2025-11-24  0.048094  If anyone used PP-OCR VL could you help me wit...
Aucun document trouvé pour cette requête.
Empty DataFrame
Columns: []
Index: []
   id                                              titre        date     score                  